In [ ]:
#!pip install pandas
!pip install scikit-learn
!pip install tensorflow-cpu
!pip install numpy pandas keras scikit-learn matplotlib scikeras

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import warnings
from datetime import datetime
from contextlib import closing


# 불필요한 경고 메시지를 숨깁니다.
warnings.filterwarnings('ignore')

print(f'completed to load library. [{datetime.now()}]')

completed to load library. [2025-06-12 08:40:08.358247]


In [2]:
def load_data_by_db(db_file_path, last_round, length, reverse=False):
    import sqlite3
    from contextlib import closing
    with closing(sqlite3.connect(db_file_path)) as conn:
        with closing(conn.cursor()) as cur:
            results = []
            first_bonus = 0
            query = f'select * from results where round>={last_round-length} and round <={last_round} '
            query += "order by round desc" if reverse else "order by round asc"
            datas = cur.execute(query).fetchall()
            for data in datas:
                if first_bonus == 0:
                    first_bonus = data[2]
                results.append((data[1], [int(i) for i in data[2].split(',')], data[3]))
            return results, first_bonus


def create_randomforest_db(db_file_path, verbose=1):
    columns = ["id INTEGER PRIMARY KEY AUTOINCREMENT",
               "n_estimator INTEGER",
               "data_length INTEGER",
               "random_state INTEGER",
               "rounds TEXT",
               "matched_cnts TEXT",
               "sum_val INTEGER",
               "version TEXT"]
    # (n_estimator, data_length, random_state, predict_lens[n_estimator][data_length][random_state], sum_val)
    query = f'CREATE TABLE IF NOT EXISTS rndforest ({",".join(columns)})'
    if verbose > 0:
        print('query=', query)
    import sqlite3
    from contextlib import closing
    with closing(sqlite3.connect(db_file_path)) as conn:
        conn.execute(query)


"""
(25, 400, 2400, [[1176, 1175, 1174, 1173], [1, 1, 3, 2]], 7)
"""
def insert_randomforest_db(db_file_path, version, db_datas, auto_commit=True, verbose=0):
    """ insert_randomforest_db """
    import sqlite3
    from contextlib import closing
    from datetime import datetime
    if version == 0:
        version = str(datetime.now().timestamp())
    with closing(sqlite3.connect(db_file_path)) as conn:
        col_dicts = {"n_estimator": db_datas[0],
                     "data_length": db_datas[1],
                     "random_state": db_datas[2],
                     "rounds": ",".join([str(i) for i in db_datas[3][0]]),
                     "matched_cnts": ",".join([str(i) for i in db_datas[3][1]]),
                     "sum_val": db_datas[4],
                     "version": version
                     }
        columns = col_dicts.keys()
        values=list(col_dicts.values())
        column_val=["?" for i in range(len(columns))]
        query = f"INSERT INTO rndforest ({','.join(columns)}) "\
                f"VALUES ({','.join(column_val)})"
        with closing(conn.cursor()) as cursor:
            if verbose > 0:
                print(f'query={query}')
                print('-'*30)
            cursor.execute(query, values)
        if auto_commit:
            conn.commit()


In [3]:
created_db = True
if created_db == False:
    create_randomforest_db('../db/metrics.db')

In [4]:
def predict(train_X, n_estimators=100, random_state=350, verbose=0):
    # 데이터를 Pandas DataFrame으로 변환합니다.
    df = pd.DataFrame(train_X, columns=['회차', '번호1', '번호2', '번호3', '번호4', '번호5', '번호6'])
    df = df.sort_values(by='회차').reset_index(drop=True)

    if verbose > 0:
        print("--- 로또 당첨 번호 이력 ---")
        print(df)

    # 기계 학습을 위한 데이터 준비
    # 각 회차의 당첨 번호(X)와 바로 다음 회차의 당첨 번호(y)를 사용합니다.
    X = df[['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']]
    y = df[['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']].shift(-1)

    # 마지막 행은 다음 회차 데이터가 없으므로 제거합니다.
    X = X[:-1]
    y = y.dropna()
    
    if verbose > 0:
        print("\n--- 훈련 데이터 (X) ---")
        print(X.head())
        print("\n--- 정답 데이터 (y) ---")
        print(y.head())

    # 머신러닝 모델 선택 및 훈련
    # 랜덤 포레스트 회귀 모델을 사용합니다.
    # the random_state parameter is used to control the randomness of the algorithm, ensuring reproducibility of results. 
    # the n_estimators parameter specifies the number of decision trees in the forest. 
    model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state) # n_estimators: 만들 트리의 개수
    model.fit(X, y)

    # 예측할 회차의 이전 회차 데이터 (1173회차)
    last_draw = df.iloc[-1][['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']].values.reshape(1, -1)

    if verbose > 0:
        print(f"\n--- {df.iloc[-1]['회차']}회차 데이터로 다음 회차 예측 ---")
        print(last_draw)

    # 다음 번호 예측
    predicted_numbers_set = []
    trial = 5
    for i in range(trial):
        predicted_numbers_float = model.predict(last_draw)
        # 예측된 번호 처리
        # 1. 소수점을 반올림하여 정수로 만듭니다.
        # 2. 1~45 사이의 값으로 보정합니다.
        # 3. 중복된 번호를 제거하고 6개를 선택합니다.
        predicted_numbers = set()
        for num in predicted_numbers_float[0]:
            # 반올림하여 정수로 변환
            int_num = int(round(num))
            # 1보다 작으면 1로, 45보다 크면 45로 보정
            if int_num < 1:
                int_num = 1
            elif int_num > 45:
                int_num = 45
            predicted_numbers.add(int_num)
        # 중복 제거 후 6개가 안되면, 부족한 만큼 다른 번호로 채웁니다.
        # (여기서는 가장 빈도가 높은 번호들 중 예측되지 않은 번호를 추가하는 방식을 사용)
        if len(predicted_numbers) < 6:
            all_numbers = df[['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']].values.flatten()
            counts = pd.Series(all_numbers).value_counts()
    
            extra_needed = 6 - len(predicted_numbers)
            for num in counts.index:
                if extra_needed == 0:
                    break
                if num not in predicted_numbers:
                    predicted_numbers.add(num)
                    extra_needed -= 1
        # 최종 예측 번호를 정렬하여 출력
        final_prediction = sorted(list(predicted_numbers))[:6]
        predicted_numbers_set.append(final_prediction)
    return predicted_numbers_set


def test_prediction(round_nums,
                    actual_numbers,
                    bonus,
                    predicted_numbers_set,
                    n_estimators=500,
                    random_state=500,
                    only_one=True,
                    verbose=0):
    if verbose>0:
        print("======================================")
        print(f"\n--- 참고: {round_nums}회 실제 번호 ---")
        print(f"번호: {actual_numbers}, 보너스: {bonus}")

    matched_len = 0

    for final_prediction in predicted_numbers_set:
        # 예측 결과와 실제 결과 비교
        matching_numbers = set(final_prediction).intersection(set(actual_numbers))
        if verbose > 0:
            print(f"\n--- 예측과 실제 결과 비교 ---")
            print(f"일치하는 번호: {sorted(list(matching_numbers))}")
            print(f"일치 개수: {len(matching_numbers)}")
            print(F"예측 번호: {final_prediction}")
            print(f"n_estimators: {n_estimators}")
            print(f"random_state: {random_state}")
        matched_len = len(matching_numbers)
        if only_one:
            break
    return matched_len


def list_to_dict(item_lists):
    dicts = {}
    for item in item_lists:
        dicts[item[0]] = (item[1], item[2])
    return dicts


print(f'completed to define. #1 [{datetime.now()}]')

completed to define. #1 [2025-06-12 08:40:12.145362]


In [5]:
verbose = 1

# 실제 번호
actual_numbers_or, _ = load_data_by_db(db_file_path='../db/metrics.db',
                                 last_round=1176,
                                 length=10, reverse=True)
actual_numbers = [actual_sets[1] for actual_sets in actual_numbers_or]

if verbose > 0:
    print('actual_numbers', type(actual_numbers), actual_numbers)

data_list_cnt = 3
data_length = 25
results_set=[]

print(f'completed to set env. [{datetime.now()}]')

actual_numbers <class 'list'> [[3, 4, 6, 8, 32, 42], [8, 11, 14, 17, 36, 39], [1, 5, 18, 20, 30, 35], [7, 9, 24, 40, 42, 44], [3, 6, 7, 11, 12, 17], [3, 13, 28, 34, 38, 42], [5, 12, 24, 26, 39, 42], [9, 21, 24, 30, 33, 37], [8, 23, 31, 35, 39, 40], [14, 23, 25, 27, 29, 42]]
completed to set env. [2025-06-12 08:40:17.414412]


**Notice**
- the random_state parameter is used to control the randomness of the algorithm, ensuring reproducibility of results. 
- the n_estimators parameter specifies the number of decision trees in the forest. This parameter is crucial as it directly impacts the model's performance and computational cost.

In [6]:
def predit_and_test_parts(round, bonus, h_data, n_estimators, random_state, verbose=0):
    """ predit_and_test_parts """
    train_X = []
    test_X = []
    for i in range(len(h_data) - 1):
        train_X.append(h_data[i][1])
    test_X = h_data[len(h_data) - 1][1]
    predicted_numbers_set = predict(train_X=train_X,
                                    n_estimators=n_estimators,
                                    random_state=random_state,
                                    verbose=verbose)
    predict_len1=test_prediction(round,
                                 test_X,
                                 bonus,
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state,
                                 verbose)
    return predicted_numbers_set, predict_len1


def predict_and_test(round, bonus, h_data, n_estimators, random_state, verbose=0):
    """ predict_and_test """
    _, predict_len = predit_and_test_parts(round, bonus, h_data, n_estimators, random_state, 0)
    return predict_len


print(f'completed to define. #2 [{datetime.now()}]')

completed to define. #2 [2025-06-12 08:40:20.420583]


In [7]:
results_set = []

print(f'completed to reset results_set. #2 [{datetime.now()}]')

completed to reset results_set. #2 [2025-06-12 08:40:25.579504]


In [21]:
### Prepare and train, predict
def main_process(version,
                 n_estimators,
                 last_rounds,
                 data_lengths,
                 random_state_gap=100,
                 random_state_begin=0,
                 random_state_end=300000,
                 db_file_path='../db/metrics.db',
                 write_to_db=False,
                 write_db_file_path='../db/metrics.db',
                 verbose=0):
    predict_lens = {}
    for n_estimator in n_estimators:
        predict_lens[n_estimator] = {}
        if n_estimator > 1:
            for last_round in last_rounds:
                cnt = 0
                for data_length in data_lengths:
                    hist_data, first_bonus = load_data_by_db(db_file_path=db_file_path, last_round=last_round, length=data_length)
                    for train_x in hist_data:
                        train_x[1].insert(0, train_x[0])
                    if data_length not in predict_lens[n_estimator]:
                        predict_lens[n_estimator][data_length] = {}
                    for random_state in range(random_state_begin, random_state_end, random_state_gap):
                        predict_len = predict_and_test(last_round, first_bonus, hist_data, n_estimator, random_state, 0)
                        if n_estimator not in predict_lens:
                            predict_lens[n_estimator] = {}
                        if random_state not in predict_lens[n_estimator][data_length]:
                            predict_lens[n_estimator][data_length][random_state] = [[last_round], [predict_len]]
                        else:
                            predict_lens[n_estimator][data_length][random_state][0].append(last_round)
                            predict_lens[n_estimator][data_length][random_state][1].append(predict_len)
                        cnt += 1
                        if cnt % 1000 == 0:
                            print(f'proceed [now={datetime.now()}, n_estimator={n_estimator}, cnt={cnt}, last_round={last_round}, random_state={random_state}, data_length={data_length}]')
    return predict_lens

print(f'completed to define main_process. #7 [{datetime.now()}]')

completed to define main_process. #7 [2025-06-12 11:07:07.588050]


In [25]:
def print_predicts(predict_lens, sum_min=-1, sum_max=-1, verbose=0, write_to_file=False, write_to_db=False):
    result_set = []
    for n_estimator in predict_lens:
        for data_length in predict_lens[n_estimator]:
            for random_state in predict_lens[n_estimator][data_length]:
                sum_val = sum(predict_lens[n_estimator][data_length][random_state][1])
                if sum_min == -1 and sum_max == -1:
                    if verbose > 0:
                        print(n_estimator, data_length, random_state, predict_lens[n_estimator][data_length][random_state])
                    result_set.append((n_estimator, data_length, random_state, predict_lens[n_estimator][data_length][random_state], sum_val))
                else:
                    if sum_min > 0 and sum_max > 0 and sum_val >= sum_min and sum_val <= sum_max:
                        if verbose > 0:
                            print(n_estimator, data_length, random_state, predict_lens[n_estimator][data_length][random_state])
                        result_set.append((n_estimator, data_length, random_state, predict_lens[n_estimator][data_length][random_state], sum_val))
                    else:
                        if sum_min > 0 and sum_val >= sum_min:
                            if verbose > 0:
                                print(n_estimator, data_length, random_state, predict_lens[n_estimator][data_length][random_state])
                            result_set.append((n_estimator, data_length, random_state, predict_lens[n_estimator][data_length][random_state], sum_val))
                        elif sum_max > 0 and sum_val <= sum_max:
                            if verbose > 0:
                                print(n_estimator, data_length, random_state, predict_lens[n_estimator][data_length][random_state])
                            result_set.append((n_estimator, data_length, random_state, predict_lens[n_estimator][data_length][random_state], sum_val))
                if write_to_file:
                    with closing(open("metric_temp.txt", "at")) as fd:
                        fd.write(f'{n_estimator},{data_length},{random_state},')
                        fd.write(f'{"/".join([str(i) for i in predict_lens[n_estimator][data_length][random_state][0]])},')
                        fd.write(f'{"/".join([str(i) for i in predict_lens[n_estimator][data_length][random_state][0]])},')
                        fd.write(f'{sum_val}\n')
                if write_to_db:
                    db_datas = (n_estimator, data_length, random_state, predict_lens[n_estimator][data_length][random_state], sum_val)
                    insert_randomforest_db('../db/metrics.db',
                                           version=version,
                                           db_datas=db_datas,
                                           verbose=verbose)


    return result_set


print(f'completed to define print_predicts. #8 [{datetime.now()}]')

completed to define print_predicts. #8 [2025-06-12 11:11:43.832917]


In [30]:
def main(parameters, version, sum_min=-1, sum_max=-1, write_to_file=False, write_to_db=False, verbose=0):
    print(f'start   [now={datetime.now()}]')    
    predict_lens = main_process(version=version,
                                n_estimators=parameters["n_estimators"],
                                last_rounds=parameters["last_rounds"],
                                data_lengths=parameters["data_lengths"],
                                random_state_gap=parameters["random_state_gap"],
                                random_state_begin=parameters["random_state_begin"],
                                random_state_end=parameters["random_state_end"],
                                verbose=verbose)
    print(f'completed [now={datetime.now()}]')
    print(f'start to read and write data: [now={datetime.now()}]')
    result_set = print_predicts(predict_lens=predict_lens,
                                sum_min=sum_min,
                                sum_max=sum_max,
                                write_to_file=write_to_file,
                                write_to_db=write_to_db)
    print(f'complete to read and data: [now={datetime.now()}]')
    return result_set

print(f'completed to define main. #9 [{datetime.now()}]')

completed to define main. #9 [2025-06-12 11:24:49.602772]


In [ ]:
parameters = {
    "n_estimators": [25, 50, 10],
    "data_lengths": [40],
    "random_state_gap": 1,
    "random_state_begin": 0,
    "random_state_end": 1000,
    "last_rounds": [1175, 1174, 1173],
}
version="T_01_02"
result_sets = main(parameters=parameters, sum_min=2, version=version)

In [28]:
parameters = {
    "n_estimators": [25, 10],
    "data_lengths": [40],
    "random_state_gap": 1,
    "random_state_begin": 0,
    "random_state_end": 1000,
    "last_rounds": [1175, 1174, 1173, 1172, 1171, 1170, 1169, 1168, 1167, 1166],
}
version="T_01_03"
result_sets = main(parameters=parameters, sum_min=2, version=version)

start   [now=2025-06-11 23:05:14.736151]
proceed [now=2025-06-11 23:05:37.338800, cnt=1000, last_round=1176, random_state=999, data_length=40]
proceed [now=2025-06-11 23:06:03.904516, cnt=1000, last_round=1175, random_state=999, data_length=40]
proceed [now=2025-06-11 23:06:29.714422, cnt=1000, last_round=1174, random_state=999, data_length=40]
proceed [now=2025-06-11 23:06:55.629254, cnt=1000, last_round=1173, random_state=999, data_length=40]
proceed [now=2025-06-11 23:07:18.650509, cnt=1000, last_round=1172, random_state=999, data_length=40]
proceed [now=2025-06-11 23:07:44.495763, cnt=1000, last_round=1171, random_state=999, data_length=40]
proceed [now=2025-06-11 23:08:10.484503, cnt=1000, last_round=1170, random_state=999, data_length=40]
proceed [now=2025-06-11 23:08:33.761643, cnt=1000, last_round=1169, random_state=999, data_length=40]
proceed [now=2025-06-11 23:08:59.960513, cnt=1000, last_round=1168, random_state=999, data_length=40]
proceed [now=2025-06-11 23:09:14.003625, 

In [36]:
parameters = {
    "n_estimators": [25, 10],
    "data_lengths": [40],
    "random_state_gap": 1,
    "random_state_begin": 0,
    "random_state_end": 1000,
    "last_rounds": [1176, 1175, 1174],
}
version="T_01_04"
result_sets = main(parameters=parameters, sum_min=2, version=version)

start   [now=2025-06-11 23:20:22.639013]
proceed [now=2025-06-11 23:20:48.674119, n_estimator=25, cnt=1000, last_round=1176, random_state=999, data_length=40]
proceed [now=2025-06-11 23:21:14.995542, n_estimator=25, cnt=1000, last_round=1175, random_state=999, data_length=40]
proceed [now=2025-06-11 23:21:37.827796, n_estimator=25, cnt=1000, last_round=1174, random_state=999, data_length=40]
proceed [now=2025-06-11 23:21:52.514248, n_estimator=10, cnt=1000, last_round=1176, random_state=999, data_length=40]
proceed [now=2025-06-11 23:22:03.630452, n_estimator=10, cnt=1000, last_round=1175, random_state=999, data_length=40]
proceed [now=2025-06-11 23:22:14.851538, n_estimator=10, cnt=1000, last_round=1174, random_state=999, data_length=40]
completed [now=2025-06-11 23:22:14.851758]


In [42]:
parameters = {
    "n_estimators": [25, 10],
    "data_lengths": [40],
    "random_state_gap": 1,
    "random_state_begin": 0,
    "random_state_end": 1000,
    "last_rounds": [1175, 1175],
}
version="T_01_05"
result_sets = main(parameters=parameters, sum_min=2, version=version)

start   [now=2025-06-11 23:33:51.682807]
proceed [now=2025-06-11 23:34:18.700060, n_estimator=25, cnt=1000, last_round=1176, random_state=999, data_length=40]
proceed [now=2025-06-11 23:34:42.910211, n_estimator=25, cnt=1000, last_round=1175, random_state=999, data_length=40]
proceed [now=2025-06-11 23:34:58.457405, n_estimator=10, cnt=1000, last_round=1176, random_state=999, data_length=40]
proceed [now=2025-06-11 23:35:11.968236, n_estimator=10, cnt=1000, last_round=1175, random_state=999, data_length=40]
completed [now=2025-06-11 23:35:11.968854]
start to read data: [now=2025-06-11 23:35:11.968929]
complete to read data: [now=2025-06-11 23:35:11.970196]
start to write db: [now=2025-06-11 23:35:11.970231]
complete to write db: [now=2025-06-11 23:35:34.319251]


In [ ]:
parameters = {
    "n_estimators": [25, 10],
    "data_lengths": [40],
    "random_state_gap": 1,
    "random_state_begin": 0,
    "random_state_end": 1000,
    "last_rounds": [1175],
}
version="T_01_06"
result_sets = main(parameters=parameters, sum_min=2, version=version)

In [62]:
parameters = {
    "n_estimators": [25],
    "data_lengths": [400],
    "random_state_gap": 1,
    "random_state_begin": 0,
    "random_state_end": 3000,
    "last_rounds": [1175, 1174, 1173, 1172, 1171, 1170, 1169, 1168, 1167, 1166],
}
version="T_01_07"
result_sets = main(parameters=parameters, sum_min=2, version=version)

start   [now=2025-06-12 00:18:32.976156]
proceed [now=2025-06-12 00:19:31.586459, n_estimator=25, cnt=1000, last_round=1176, random_state=999, data_length=400]
proceed [now=2025-06-12 00:20:31.094794, n_estimator=25, cnt=2000, last_round=1176, random_state=1999, data_length=400]
proceed [now=2025-06-12 00:21:30.204987, n_estimator=25, cnt=3000, last_round=1176, random_state=2999, data_length=400]
proceed [now=2025-06-12 00:22:28.500770, n_estimator=25, cnt=1000, last_round=1175, random_state=999, data_length=400]
proceed [now=2025-06-12 00:23:29.680448, n_estimator=25, cnt=2000, last_round=1175, random_state=1999, data_length=400]
proceed [now=2025-06-12 00:24:30.779908, n_estimator=25, cnt=3000, last_round=1175, random_state=2999, data_length=400]
proceed [now=2025-06-12 00:25:27.806711, n_estimator=25, cnt=1000, last_round=1174, random_state=999, data_length=400]
proceed [now=2025-06-12 00:26:23.534800, n_estimator=25, cnt=2000, last_round=1174, random_state=1999, data_length=400]
pr

In [32]:
parameters = {
    "n_estimators": [25,10],
    "data_lengths": [400],
    "random_state_gap": 10,
    "random_state_begin": 300000,
    "random_state_end": 400000,
    "last_rounds": [1175, 1174, 1173, 1172, 1171, 1170, 1169, 1168, 1167, 1166],
}
version="T_01_08"
result_sets = main(parameters=parameters, version=version, sum_min=0, sum_max=-1, write_to_db=True)

start   [now=2025-06-12 11:27:32.328992]
proceed [now=2025-06-12 11:28:35.492393, n_estimator=25, cnt=1000, last_round=1175, random_state=309990, data_length=400]
proceed [now=2025-06-12 11:29:38.640119, n_estimator=25, cnt=2000, last_round=1175, random_state=319990, data_length=400]
proceed [now=2025-06-12 11:30:41.805702, n_estimator=25, cnt=3000, last_round=1175, random_state=329990, data_length=400]
proceed [now=2025-06-12 11:31:45.467498, n_estimator=25, cnt=4000, last_round=1175, random_state=339990, data_length=400]
proceed [now=2025-06-12 11:32:48.559776, n_estimator=25, cnt=5000, last_round=1175, random_state=349990, data_length=400]
proceed [now=2025-06-12 11:33:51.756570, n_estimator=25, cnt=6000, last_round=1175, random_state=359990, data_length=400]
proceed [now=2025-06-12 11:34:55.399876, n_estimator=25, cnt=7000, last_round=1175, random_state=369990, data_length=400]
proceed [now=2025-06-12 11:35:58.458378, n_estimator=25, cnt=8000, last_round=1175, random_state=379990, 

In [36]:
for result in result_sets:
    print(result)

In [ ]:
parameters = {
    "n_estimators": [25,10],
    "data_lengths": [400],
    "random_state_gap": 10,
    "random_state_begin": 100000,
    "random_state_end": 150000,
    "last_rounds": [1175, 1174, 1173, 1172, 1171, 1170, 1169, 1168, 1167, 1166],
}
version="T_01_08"
result_sets = main(parameters=parameters, version=version, sum_min=0, sum_max=-1, write_to_db=True)

start   [now=2025-06-12 14:40:37.246073]
proceed [now=2025-06-12 14:41:42.718049, n_estimator=25, cnt=1000, last_round=1175, random_state=109990, data_length=400]
proceed [now=2025-06-12 14:42:47.658780, n_estimator=25, cnt=2000, last_round=1175, random_state=119990, data_length=400]
proceed [now=2025-06-12 14:43:50.521634, n_estimator=25, cnt=3000, last_round=1175, random_state=129990, data_length=400]
proceed [now=2025-06-12 14:44:52.960051, n_estimator=25, cnt=4000, last_round=1175, random_state=139990, data_length=400]
proceed [now=2025-06-12 14:45:55.507010, n_estimator=25, cnt=5000, last_round=1175, random_state=149990, data_length=400]
proceed [now=2025-06-12 14:46:58.081716, n_estimator=25, cnt=1000, last_round=1174, random_state=109990, data_length=400]
proceed [now=2025-06-12 14:48:00.428461, n_estimator=25, cnt=2000, last_round=1174, random_state=119990, data_length=400]
proceed [now=2025-06-12 14:49:02.731608, n_estimator=25, cnt=3000, last_round=1174, random_state=129990, 

In [31]:
parameters = {
    "n_estimators": [25,10],
    "data_lengths": [400],
    "random_state_gap": 1,
    "random_state_begin": 300000,
    "random_state_end": 301000,
    "last_rounds": [1175],
}
version="T_01_99"
result_sets = main(parameters=parameters, version=version, sum_min=0, sum_max=-1, write_to_db=True)

start   [now=2025-06-12 11:24:56.202755]
proceed [now=2025-06-12 11:25:59.665769, n_estimator=25, cnt=1000, last_round=1175, random_state=300999, data_length=400]
proceed [now=2025-06-12 11:26:29.527030, n_estimator=10, cnt=1000, last_round=1175, random_state=300999, data_length=400]
completed [now=2025-06-12 11:26:29.527225]
completed [now=2025-06-12 11:26:29.527241]
start to read data: [now=2025-06-12 11:26:29.527249]
complete to read data: [now=2025-06-12 11:26:54.347066]
start to write db: [now=2025-06-12 11:26:54.347180]
complete to write db: [now=2025-06-12 11:26:54.347199]


In [51]:
## SQL Query

In [46]:
!echo "select * from rndforest where version=\"T_01_03\" and random_state=762;" | sqlite3 ../db/metrics.db
!echo "........................................."
!echo "select * from rndforest where version=\"T_01_03\" and sum_val > 12;" | sqlite3 ../db/metrics.db

11784|25|40|762|1176,1175,1174,1173,1172,1171,1170,1169,1168|1,0,0,0,0,1,1,1,1|5|T_01_03
12783|10|40|762|1176,1175,1174,1173,1172,1171,1170,1169,1168|3,2,1,0,0,1,2,0,1|10|T_01_03
.........................................
11200|25|40|178|1176,1175,1174,1173,1172,1171,1170,1169,1168|2,2,1,1,2,0,1,2,2|13|T_01_03
11287|25|40|265|1176,1175,1174,1173,1172,1171,1170,1169,1168|1,2,1,3,2,1,1,1,2|14|T_01_03
11326|25|40|304|1176,1175,1174,1173,1172,1171,1170,1169,1168|0,0,1,2,2,1,2,4,1|13|T_01_03
11519|25|40|497|1176,1175,1174,1173,1172,1171,1170,1169,1168|0,1,2,3,2,0,2,2,2|14|T_01_03
11642|25|40|620|1176,1175,1174,1173,1172,1171,1170,1169,1168|1,1,0,3,2,1,2,2,1|13|T_01_03
11813|25|40|791|1176,1175,1174,1173,1172,1171,1170,1169,1168|2,1,2,2,2,1,1,3,0|14|T_01_03
12090|10|40|68|1176,1175,1174,1173,1172,1171,1170,1169,1168|1,0,0,3,1,1,2,3,2|13|T_01_03
12119|10|40|97|1176,1175,1174,1173,1172,1171,1170,1169,1168|1,1,0,3,1,1,2,1,3|13|T_01_03
12285|10|40|264|1176,1175,1174,1173,1172,1171,1170,1169,1168|

In [ ]:
old_versions = ["T_01_01", "T_01_02", "T_01_03", "T_01_04", "T_01_05"]
sum_val = 7
for version in old_versions:
    !echo "select * from rndforest where version=\"{version}\" and sum_val > {sum_val};" | sqlite3 ../db/metrics.db

In [35]:
# find n_estimators=[25, 10]
#  data_lengths = [40]
#  random_state_gap = 1
#  last_rounds = [1176, 1175, 1174, 1173]
!echo "select * from rndforest where version=\"T_01_02\" and sum_val > 7;" | sqlite3 ../db/metrics.db

8517|25|40|430|1176,1175,1174,1173|2,1,1,4|8|T_01_02
10170|10|40|129|1176,1175,1174,1173|2,2,2,2|8|T_01_02


In [41]:
# find n_estimators=[25, 10]
#  data_lengths = [40]
#  random_state_gap = 1
#  last_rounds = [1176, 1175, 1174]
!echo "select * from rndforest where version=\"T_01_04\" and sum_val > 5;" | sqlite3 ../db/metrics.db

13998|10|40|129|1176,1175,1174|2,2,2|6|T_01_04
14122|10|40|264|1176,1175,1174|3,1,2|6|T_01_04
14282|10|40|439|1176,1175,1174|2,2,2|6|T_01_04
14576|10|40|762|1176,1175,1174|3,2,1|6|T_01_04


In [44]:
# find n_estimators=[25, 10]
#  data_lengths = [40]
#  random_state_gap = 1
#  last_rounds = [1176, 1175]
version="T_01_05"
!echo "select * from rndforest where version=\"T_01_05\" and sum_val > 3;" | sqlite3 ../db/metrics.db

14802|25|40|7|1176,1175|2,2|4|T_01_05
14895|25|40|133|1176,1175|2,2|4|T_01_05
14910|25|40|158|1176,1175|2,2|4|T_01_05
14921|25|40|178|1176,1175|2,2|4|T_01_05
14932|25|40|194|1176,1175|2,2|4|T_01_05
15280|25|40|702|1176,1175|2,2|4|T_01_05
15530|10|40|59|1176,1175|1,3|4|T_01_05
15575|10|40|129|1176,1175|2,2|4|T_01_05
15642|10|40|214|1176,1175|2,2|4|T_01_05
15686|10|40|264|1176,1175|3,1|4|T_01_05
15721|10|40|304|1176,1175|2,2|4|T_01_05
15730|10|40|314|1176,1175|2,2|4|T_01_05
15735|10|40|319|1176,1175|2,2|4|T_01_05
15762|10|40|357|1176,1175|3,1|4|T_01_05
15800|10|40|405|1176,1175|2,2|4|T_01_05
15807|10|40|412|1176,1175|2,2|4|T_01_05
15826|10|40|439|1176,1175|2,2|4|T_01_05
15870|10|40|494|1176,1175|2,2|4|T_01_05
15926|10|40|566|1176,1175|2,2|4|T_01_05
15939|10|40|585|1176,1175|2,2|4|T_01_05
16032|10|40|702|1176,1175|2,2|4|T_01_05
16082|10|40|762|1176,1175|3,2|5|T_01_05
16115|10|40|800|1176,1175|2,2|4|T_01_05
16219|10|40|933|1176,1175|2,2|4|T_01_05


In [57]:
# find n_estimators=[25, 10]
#  data_lengths = [40]
#  random_state_gap = 1
#  last_rounds = [1176]
version="T_01_05"
!echo "select * from rndforest where version=\"T_01_03\" and random_state in (357, 264, 762, 864);" | sqlite3 ../db/metrics.db
!echo "select * from rndforest where version=\"T_01_06\" and sum_val > 2;" | sqlite3 ../db/metrics.db

11286|25|40|264|1176,1175,1174,1173,1172,1171,1170,1169,1168|1,0,0,1,2,0,1,1,1|7|T_01_03
11379|25|40|357|1176,1175,1174,1173,1172,1171,1170,1169,1168|0,0,1,1,1,1,0,1,0|5|T_01_03
11784|25|40|762|1176,1175,1174,1173,1172,1171,1170,1169,1168|1,0,0,0,0,1,1,1,1|5|T_01_03
11886|25|40|864|1176,1175,1174,1173,1172,1171,1170,1169,1168|0,0,0,2,2,0,1,0,1|6|T_01_03
12285|10|40|264|1176,1175,1174,1173,1172,1171,1170,1169,1168|3,1,2,1,2,1,1,1,1|13|T_01_03
12378|10|40|357|1176,1175,1174,1173,1172,1171,1170,1169,1168|3,1,0,1,0,1,1,1,0|8|T_01_03
12783|10|40|762|1176,1175,1174,1173,1172,1171,1170,1169,1168|3,2,1,0,0,1,2,0,1|10|T_01_03
12885|10|40|864|1176,1175,1174,1173,1172,1171,1170,1169,1168|3,0,1,1,1,0,0,1,0|7|T_01_03
.........................................
16844|10|40|264|1176|3|3|T_01_06
16894|10|40|357|1176|3|3|T_01_06
17122|10|40|762|1176|3|3|T_01_06
17184|10|40|864|1176|3|3|T_01_06


In [70]:
# find n_estimators=[25, 10]
#  data_lengths = [40]
#  random_state_gap = 1
#  last_rounds = [1176]
!echo "select * from rndforest where version=\"T_01_03\" and sum_val > 14;" | sqlite3 ../db/metrics.db
!echo "...........like 4.............................."
!echo "select * from rndforest where version in (\"T_01_03\", \"T_01_07\") and matched_cnts like \"%4%\";" | sqlite3 ../db/metrics.db
!echo "...........like 5.............................."
!echo "select * from rndforest where version in (\"T_01_03\", \"T_01_07\") and matched_cnts like \"%5%\";" | sqlite3 ../db/metrics.db
!echo "...........like 6.............................."
!echo "select * from rndforest where version in (\"T_01_03\", \"T_01_07\") and matched_cnts like \"%6%\";" | sqlite3 ../db/metrics.db

12325|10|40|304|1176,1175,1174,1173,1172,1171,1170,1169,1168|2,2,0,2,1,1,2,4,1|15|T_01_03
...........like 4..............................
11072|25|40|50|1176,1175,1174,1173,1172,1171,1170,1169,1168|0,1,1,4,0,0,2,0,0|8|T_01_03
11204|25|40|182|1176,1175,1174,1173,1172,1171,1170,1169,1168|2,0,0,4,0,0,0,1,1|8|T_01_03
11258|25|40|236|1176,1175,1174,1173,1172,1171,1170,1169,1168|2,1,0,4,0,0,1,1,0|9|T_01_03
11326|25|40|304|1176,1175,1174,1173,1172,1171,1170,1169,1168|0,0,1,2,2,1,2,4,1|13|T_01_03
11335|25|40|313|1176,1175,1174,1173,1172,1171,1170,1169,1168|1,1,0,4,0,1,0,1,0|8|T_01_03
11399|25|40|377|1176,1175,1174,1173,1172,1171,1170,1169,1168|0,1,1,4,1,0,1,1,0|9|T_01_03
11452|25|40|430|1176,1175,1174,1173,1172,1171,1170,1169,1168|2,1,1,4,0,0,1,2,0|11|T_01_03
11508|25|40|486|1176,1175,1174,1173,1172,1171,1170,1169,1168|1,0,1,4,1,1,0,0,0|8|T_01_03
11564|25|40|542|1176,1175,1174,1173,1172,1171,1170,1169,1168|0,1,0,1,2,1,2,4,0|11|T_01_03
11691|25|40|669|1176,1175,1174,1173,1172,1171,1170,1169,116

In [67]:
!echo "select * from rndforest where version in (\"T_01_07\", \"T_01_03\") and sum_val > 13;" | sqlite3 ../db/metrics.db

11287|25|40|265|1176,1175,1174,1173,1172,1171,1170,1169,1168|1,2,1,3,2,1,1,1,2|14|T_01_03
11519|25|40|497|1176,1175,1174,1173,1172,1171,1170,1169,1168|0,1,2,3,2,0,2,2,2|14|T_01_03
11813|25|40|791|1176,1175,1174,1173,1172,1171,1170,1169,1168|2,1,2,2,2,1,1,3,0|14|T_01_03
12325|10|40|304|1176,1175,1174,1173,1172,1171,1170,1169,1168|2,2,0,2,1,1,2,4,1|15|T_01_03
17278|25|400|23|1176,1175,1174,1173,1172,1171,1170,1169,1168|1,0,3,1,3,2,0,3,1|14|T_01_07
17342|25|400|87|1176,1175,1174,1173,1172,1171,1170,1169,1168|2,1,3,1,1,1,2,2,1|14|T_01_07
17449|25|400|194|1176,1175,1174,1173,1172,1171,1170,1169,1168|1,1,2,2,2,2,0,3,2|15|T_01_07
17566|25|400|311|1176,1175,1174,1173,1172,1171,1170,1169,1168|1,1,3,1,1,2,3,2,1|15|T_01_07
17890|25|400|635|1176,1175,1174,1173,1172,1171,1170,1169,1168|2,1,3,0,1,0,1,4,2|14|T_01_07
17935|25|400|680|1176,1175,1174,1173,1172,1171,1170,1169,1168|1,1,1,1,1,2,1,4,4|16|T_01_07
18167|25|400|912|1176,1175,1174,1173,1172,1171,1170,1169,1168|1,1,3,1,3,1,0,3,1|14|T_01_07
18384

In [146]:
for result in result_set:
    if result[4] > 6:
        print(result)

(25, 400, 670, [[1176, 1175, 1174, 1173], [2, 2, 0, 3]], 7)
(25, 400, 1210, [[1176, 1175, 1174, 1173], [1, 1, 3, 2]], 7)
(25, 400, 2190, [[1176, 1175, 1174, 1173], [1, 2, 2, 2]], 7)
(25, 400, 2400, [[1176, 1175, 1174, 1173], [1, 1, 3, 2]], 7)
(25, 400, 4900, [[1176, 1175, 1174, 1173], [1, 2, 3, 1]], 7)
(25, 400, 5480, [[1176, 1175, 1174, 1173], [2, 1, 3, 1]], 7)
(25, 400, 5850, [[1176, 1175, 1174, 1173], [2, 1, 3, 1]], 7)
(25, 400, 6370, [[1176, 1175, 1174, 1173], [2, 1, 3, 1]], 7)
(25, 400, 6680, [[1176, 1175, 1174, 1173], [2, 1, 3, 2]], 8)
(25, 400, 8350, [[1176, 1175, 1174, 1173], [1, 1, 3, 2]], 7)
(25, 400, 11270, [[1176, 1175, 1174, 1173], [1, 2, 3, 1]], 7)
(25, 400, 11920, [[1176, 1175, 1174, 1173], [2, 1, 1, 3]], 7)
(25, 400, 12170, [[1176, 1175, 1174, 1173], [1, 2, 2, 2]], 7)
(25, 400, 12230, [[1176, 1175, 1174, 1173], [1, 1, 3, 2]], 7)
(25, 400, 13220, [[1176, 1175, 1174, 1173], [1, 1, 3, 2]], 7)
(25, 400, 14750, [[1176, 1175, 1174, 1173], [2, 1, 3, 1]], 7)
(25, 400, 16160, [[

In [13]:
print(f'check answer : [{datetime.now()}]')
for result in results_set:
    if result[2][0] > 3 or result[2][1] > 3:
        print(result)

check answer : [2025-06-09 11:35:00.484062]


In [12]:
n_estimators=45
begin_pos=0
end_pos=600000
print(f'start   [now={datetime.now()}]')
for minus_value in [2, 1, 0]:
    cnt = 0
    for random_state in range(begin_pos, end_pos, 100):
        predict_lens = predict_and_test(minus_value, n_estimators, random_state, 0)
        if predict_lens[0] > 2 and predict_lens[1] > 2:
            results_set.append((n_estimators, random_state, predict_lens, minus_value))
            print('found = ', n_estimators, random_state, predict_lens, minus_value)
        cnt += 1
        if cnt % 1000 == 0:
            print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

start   [now=2025-06-11 00:42:42.831085]
proceed [now=2025-06-11 00:44:28.745292, 1000]
proceed [now=2025-06-11 00:46:14.410815, 2000]
proceed [now=2025-06-11 00:48:00.175914, 3000]
proceed [now=2025-06-11 00:49:45.904667, 4000]
proceed [now=2025-06-11 00:51:31.441960, 5000]
proceed [now=2025-06-11 00:53:17.201224, 6000]
proceed [now=2025-06-11 00:55:03.388433, 1000]
proceed [now=2025-06-11 00:56:49.462688, 2000]
proceed [now=2025-06-11 00:58:35.198011, 3000]
proceed [now=2025-06-11 01:00:20.651806, 4000]
proceed [now=2025-06-11 01:02:04.516120, 5000]
proceed [now=2025-06-11 01:03:48.433397, 6000]
proceed [now=2025-06-11 01:05:32.586867, 1000]
proceed [now=2025-06-11 01:07:16.806856, 2000]
proceed [now=2025-06-11 01:09:01.147398, 3000]
proceed [now=2025-06-11 01:10:45.591498, 4000]
proceed [now=2025-06-11 01:12:30.120140, 5000]
proceed [now=2025-06-11 01:14:14.759978, 6000]
completed [now=2025-06-11 01:14:14.760198]


In [ ]:
n_estimators=6
begin_pos=100000
end_pos=500000
print(f'start   [now={datetime.now()}]')
for minus_value in [8, 7, 6, 5, 4, 3, 2, 1]:
    cnt = 0
    for random_state in range(begin_pos, end_pos, 100):
        predict_lens = predict_and_test(minus_value, n_estimators, random_state, 0)
        if predict_lens[0] > 1 and predict_lens[1] > 1 and predict_lens[2] > 1:
            results_set.append((n_estimators, random_state, predict_lens, minus_value))
            print('found = ', n_estimators, random_state, predict_lens, minus_value)
        cnt += 1
        if cnt % 1000 == 0:
            print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

```
(25, 219100, [3, 4], 0)
(25, 560100, [3, 4], 0)
(25, 322300, [3, 4], 5)
(25, 520500, [3, 4], 5)
(25, 148700, [3, 4], 6)
(25, 283500, [3, 4], 6)
(25, 358100, [3, 4], 6)
(25, 451600, [4, 3], 6)
(25, 489000, [4, 3], 6)
(25, 295500, [3, 4], 7)
(25, 241500, [3, 4], 8)
(45, 52100, [3, 4], 6)
(45, 587300, [3, 4], 4)
(10, 102500, [3, 4], 7)
(10, 113700, [4, 4], 6)
(10, 164200, [3, 4], 1)
(10, 225300, [3, 4], 8)
(10, 371400, [3, 4], 8)
(10, 255700, [3, 4], 7)
(10, 15200, [3, 4], 7)
(10, 63300, [3, 4], 6)
(6, 131400, [3, 4], 5)
(6, 195200, [4, 3], 5)
(6, 406100, [4, 3], 8)
(6, 325200, [4, 3], 7)
(6, 353200, [4, 3], 7)
(6, 473900, [4, 4], 6)
(6, 363000, [4, 3], 5)
[
(25, 451600, [4, 3], 6),
(25, 489000, [4, 3], 6),
(6, 473900, [4, 4], 6),
(10, 113700, [4, 4], 6),
(10, 15200, [3, 4], 7),
]

```

In [ ]:
for result in results_set:
    if result[2][0] > 3 or result[2][1] > 3:
        print(result)

In [ ]:
datas = [
    [45,208000,4,3,5],
    [45,101700,4,3,2],
    [45,101700,4,3,2],
    [45,185400,4,3,2],
    [45,96800,4,3,1],
    [45,58600,3,3,0],
    ]
datas = [
    (25, 451600, [4, 3], 6),
    (25, 489000, [4, 3], 6),
    (6, 473900, [4, 4], 6),
    (10, 113700, [4, 4], 6),
    (10, 15200, [3, 4], 7),
    ]
for data in datas:
    n_estimators=data[0]
    random_state=data[1]
    minus_value=data[3]
    results=[]
    print(f'start [now={datetime.now()}]')
    data_pos=1
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=random_state,
                                    verbose=0)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state,
                                 verbose=1)
    data_pos=2
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=random_state,
                                    verbose=0)
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[data_pos],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state,
                                   verbose=1)
print(f'completed [now={datetime.now()}]')

In [ ]:
# datas = [
#     [45,208000,4,3,5],
#     [45,101700,4,3,2],
#     [45,185400,4,3,2],
#     [45,96800,4,3,1],
#     [45,58600,3,3,0],
#     ]
datas = [
    (25, 451600, [4, 3], 6),
    (25, 489000, [4, 3], 6),
    (6, 473900, [4, 4], 6),
    (10, 113700, [4, 4], 6),
    (10, 15200, [3, 4], 7),
    ]
print(f'start [now={datetime.now()}]')
results=[]
for data in datas:
    n_estimators=data[0]
    random_state=data[1]
    minus_value=data[3]    
    data_pos=0
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=random_state,
                                    verbose=0)
    results.append(predicted_numbers_set[0])
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[1],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state,
                                   verbose=1)
print(f'completed [now={datetime.now()}]')
print('Predicted Numbers.')
for result in results:
    print(result)

In [ ]:
print(results)

```
[
[45,208000,4,3,5],
[45,101700,4,3,2],
[45,101700,4,3,2],
[45,185400,4,3,2],
[45,96800,4,3,1],
    [45,58600,3,3,0],
]
```

In [ ]:
n_estimators=10
random_state=5000
data_pos=1
h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]]
predicted_numbers_set = predict(lotto_history_data=h_data,
                                n_estimators=n_estimators,
                                random_state=random_state,
                                verbose=0
                               )
test_prediction(round_nums[data_pos],
                actual_numbers[data_pos],
                bonus[data_pos],
                predicted_numbers_set,
                n_estimators,
                random_state)
data_pos=2
h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]]
predicted_numbers_set = predict(lotto_history_data=h_data,
                                n_estimators=n_estimators,
                                random_state=random_state,
                                verbose=0)
test_prediction(round_nums[data_pos],
                actual_numbers[data_pos],
                bonus[data_pos],
                predicted_numbers_set,
                n_estimators,
                random_state)

In [ ]:
[
    ([14, 17, 23, 27, 34, 36], [14, 17, 36], (45, 1000), (1160, 1173),
    ([13, 16, 23, 27, 34, 36], [36], (45, 1300), (1160, 1173)),
    ([11, 15, 21, 30, 36, 39], [11, 36, 39], (45, 1000), (1160, 1173),

]